# 라이브러리 호출

In [ ]:
!pip install koreanize-matplotlib
import koreanize_matplotlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 17.9 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno  # missingno : 결측데이터를 파악하는데 직관적인 도움을 주는 패키지
%matplotlib inline
import re
# 날짜컬럼 형식 바꾸기
import datetime

# 이 두 줄의 코드는 matplotlib의 기본 scheme말고, seaborn scheme을 세팅
# 일일이 graph의 font size를 지정할 필요 없이, seaborn의 font_scale을 사용하면 편리
# plt.style.use('seaborn')
# sns.set(font_scale = 2.5)

import warnings
warnings.filterwarnings('ignore')

# 그래프에서 한글 폰트 깨지는 문제를 해결해주기 위한 코드
from matplotlib import font_manager, rc
plt.rcParams['axes.unicode_minus'] = False

import platform  # 현재 운영체제를 확인

if platform.system() == 'Darwin':  # macOS(Darwin)인 경우에는 rc() 함수를 사용하여 한글 폰트를 'AppleGothic'으로 설정
  rc('font', family = 'AppleGothic')
elif platform.system() == 'Windows':
  path = 'c:Windows/Fonts/Century Gothic'
  font_name = font_manager.FontProperties(fname = path).get_name()
  rc('font', family = font_name)
else:
  plt.rc('font', family='NanumGothic')

# 데이터 셋 확인

In [ ]:
# 상세데이터 파일 불러오기

# ctd = pd.read_excel('카페원본데이터')
# visitor = pd.read_excel('22.2~23.5 관광객입도현황.xlsx')
# visitor1 = pd.read_excel('관광공사입도현황.xlsx')
# ksv = pd.read_excel('전국숙박관광객.xlsx')
# ta = pd.read_excel('해외출국현황.xls')
# pc = pd.read_excel('2202~2305_관광소비 추이_내국인.xlsx')
# dv = pd.read_excel('일별제주입도객.xlsx')

## 커피템플 상세데이터

In [ ]:
ctd.head()

In [ ]:
ctd.isnull().sum()

In [ ]:
ctd.info()

In [ ]:
ctd.describe()

### 전처리

In [ ]:
# 결제일, 결제시간이 문자형이니까 먼저 결제일+결제시간 만들기
ctd['결제ID'] = ctd['결제일']+ " " +ctd['결제시간']
ctd.head()

In [ ]:
# 결제일,시간을 날짜형식으로 바꾸기
ctd['결제일'] = pd.to_datetime(ctd['결제일'], format = "%Y-%m-%d", errors = 'raise')  # 에러발생 -> 에러발생
ctd['결제ID'] = pd.to_datetime(ctd['결제ID'], format="%Y-%m-%d %H:%M:%S", errors='raise')
ctd.info()

In [ ]:
# 연,월 컬럼 생성
ctd['연'] = ctd['결제일'].dt.year
ctd['월'] = ctd['결제일'].dt.month
ctd['요일'] = ctd['결제일'].dt.day_name()
ctd.head()

In [ ]:
# 시간대별 주문수?(상품명 전처리 필요)
ctd.groupby(ctd['결제ID'].dt.hour).size()

In [ ]:
# 시간 별 매출
ctd.groupby(ctd['결제ID'].dt.hour)['상품별 합계'].sum()

In [ ]:
# 월별 주문건수
ctd_oc = ctd.groupby(['연','월'])['결제ID'].nunique()
ctd_oc = ctd_oc.reset_index()
ctd_oc.rename(columns={'결제ID': '주문수'}, inplace=True)
ctd_oc

In [ ]:
# 월별 매출
ctd_yms = ctd.groupby(['연','월'])['상품별 합계'].sum()
ctd_yms = ctd_yms.reset_index()
ctd_yms.rename(columns={'상품별 합계': '매출'}, inplace=True)
ctd_yms

## 제주도 관광객 입도 데이터

In [ ]:
visitor.head()

In [ ]:
visitor.isnull().sum()

In [ ]:
visitor.info()

In [ ]:
visitor1.head()

In [ ]:
visitor1.info()

### 전처리

In [ ]:
# 연,월 컬럼 생성
visitor['연'] = visitor['날짜'].dt.year
visitor['월'] = visitor['날짜'].dt.month
visitor.head()

In [ ]:
# 월별 제주도 관광객
visitor_s = visitor[['연','월','총계']]
visitor_s = visitor_s.reset_index()
visitor_s.rename(columns={'총계': '관광객'}, inplace=True)
visitor_s

In [ ]:
# 매출과 월별 관광객
yms_v = pd.merge(ctd_yms, visitor_s, on=['연', '월'])
yms_v = yms_v.drop('index', axis=1)
yms_v

In [ ]:
# 월별 주문수와 관광객
ymoc_v = pd.merge(ctd_oc, visitor_s, on=['연', '월'])
ymoc_v = ymoc_v.drop('index', axis=1)
ymoc_v

In [ ]:
# 연과 월을 합쳐서 날짜 컬럼 만들기
yms_v['날짜'] = yms_v['연'].astype(str) + '-' + yms_v['월'].astype(str)
yms_v['날짜'] = pd.to_datetime(yms_v['날짜'], format='%Y-%m')
yms_v = yms_v.reset_index()
yms_v = yms_v.drop('index', axis=1)
yms_v.head()

In [ ]:
# 연과 월을 합쳐서 날짜 컬럼 만들기
ymoc_v['날짜'] = ymoc_v['연'].astype(str) + '-' + ymoc_v['월'].astype(str)
ymoc_v['날짜'] = pd.to_datetime(ymoc_v['날짜'], format='%Y-%m')
ymoc_v = ymoc_v.reset_index()
ymoc_v = ymoc_v.drop('index', axis=1)
ymoc_v.head()

## 숙박관광객 데이터, 해외출국현황, 관광객 소비 현황 데이터

In [ ]:
ksv.head()

In [ ]:
ksv.info()

In [ ]:
ta

In [ ]:
ta.info()

In [ ]:
pc.head()

In [ ]:
pc.info()

### 전처리

In [ ]:
# 월별 해외관광객
# ta 컬럼명 바꾸기
ta.rename(columns={'전체':'날짜', '인원': '해외관광객'}, inplace=True)

In [ ]:
# '날짜' 컬럼에서 연도와 월 정보를 추출하여 datetime 형식으로 변환
ta['날짜'] = pd.to_datetime(ta['날짜'], format='%Y년%m월')

In [ ]:
# 월별 해외관광객
# 연,월 컬럼 생성
ta['연'] = ta['날짜'].dt.year
ta['월'] = ta['날짜'].dt.month
ta

In [ ]:
# 월별 해외관광객, 제주관광객
v_ta = pd.merge(ta, visitor_s, on=['연', '월'])
v_ta = v_ta.drop('index', axis=1)
v_ta.head(), v_ta.info()

In [ ]:
# 업종별 관광객 소비금액
# 연,월 컬럼 생성
pc['연'] = pc['기준연월'].dt.year
pc['월'] = pc['기준연월'].dt.month
pc.head()

In [ ]:
# 관광객의 업종별 소비금액과 커피템플의 매출
yms_pc = pd.merge(ctd_yms, pc, on=['연', '월'])
yms_pc

In [ ]:
yms_pc.info()

In [ ]:
dv.info()

In [ ]:
# '일자' 열을 활용하여 요일을 추출하여 '요일','월','일' 열을 추가
dv['요일'] = dv['일자'].dt.day_name()
dv['연'] = dv['일자'].dt.year
dv['월'] = dv['일자'].dt.month
dv.head()

In [ ]:
weekday_korean = {
    'Monday': '월요일',
    'Tuesday': '화요일',
    'Wednesday': '수요일',
    'Thursday': '목요일',
    'Friday': '금요일',
    'Saturday': '토요일',
    'Sunday': '일요일'
}

dv['요일'] = dv['요일'].map(weekday_korean)

dv.head()

# 탐색적 데이터 분석(EDA)

## 커피템플 상세데이터, 입도객

In [ ]:
ctd.nunique()

In [ ]:
# 필요없는 상품명 제거(굳이 해야되나?)
# 정규표현식 패턴 설정
pattern = r'벨\d+'

# 여러 조건을 포함하여 필터링(더 있음)
ctd_n = ctd[
    (~ctd['상품명'].str.contains('종이백')) &
    (~ctd['상품명'].str.contains('창가')) &
    (~ctd['상품명'].str.contains('큰테이블')) &
    (~ctd['상품명'].str.contains('연하게')) &
    (~ctd['상품명'].str.contains(pattern, regex=True)) &
    (~ctd['상품명'].str.contains('포장')) &
    (~ctd['상품명'].str.contains('야외')) &
    (~ctd['상품명'].str.contains('적게')) &
    (~ctd['상품명'].str.contains('추가')) &
    (~ctd['상품명'].str.contains('달게')) &
    (~ctd['상품명'].str.contains('보냉백'))
]

In [ ]:
# 시간 별 주문 상품
ctd.groupby(ctd['결제ID'].dt.hour)['상품명'].value_counts()

In [ ]:
# 전체 기간 주문수 상위 20 상품명
ctd_a = ctd_n['상품명'].value_counts().head(20)
ctd_a = ctd_a.reset_index()
ctd_a.columns = ['상품명', '주문수']
ctd_a

In [ ]:
# 2월에 대해 '상품명'을 그룹화하고 갯수를 세어서 내림차순으로 정렬
ctd_222o = ctd_n[(ctd_n['연'] == 2022) & (ctd_n['월'] == 2)]['상품명'].value_counts().reset_index()
ctd_222o.columns = ['상품명', '주문수']

# 결과 출력
print(ctd_222o)
ctd_222o.to_excel('2월 상품 주문수.xlsx', index=False)

In [ ]:
# 2022년도 월별 상품 주문수 상위 10개를 저장
ctd_dict = {}

for i in range(2, 13):
    var_name = f'ctd_22{i}'
    ctd_dict[var_name] = ctd_n[(ctd_n['연'] == 2022) & (ctd_n['월'] == i)]['상품명'].value_counts().reset_index().head(10)
    ctd_dict[var_name].columns = [f'{i}월_상품명', f'{i}월_주문수']

In [ ]:
# 2023년도 월별 상품 주문수 상위 10개를 저장
ctd_dict1 = {}

for i in range(1, 6):
    var_name = f'ctd_23{i}'
    ctd_dict1[var_name] = ctd_n[(ctd_n['연'] == 2023) & (ctd_n['월'] == i)]['상품명'].value_counts().reset_index().head(10)
    ctd_dict1[var_name].columns = [f'{i}월_상품명', f'{i}월_주문수']

In [ ]:
# 시간별 상품 주문수 상위 10개를 저장 (23년 4,5월 메뉴 이름 바뀐거 때문에 전처리 다시해야됨)
ctd_dict2 = {}

for i in range(7, 20):
    var_name = f'ctd_h_{i}'
    ctd_dict2[var_name] = ctd_n[(ctd_n['결제ID'].dt.hour == i)]['상품명'].value_counts().reset_index().head(10)
    ctd_dict2[var_name].columns = [f'{i}시_상품명', f'{i}시_주문수']

In [ ]:
# 2022년도 월별 시간대별 매출
ctd_dict3 = {}

for i in range(2, 13):
    var_name = f'ctd_22s{i}'
    # '결제ID' 컬럼의 데이터를 datetime 형식으로 변환하여 시간 정보 추출 후 그룹화
    ctd_n['결제ID'] = pd.to_datetime(ctd_n['결제ID'])
    monthly_sales = ctd_n[(ctd_n['연'] == 2022) & (ctd_n['월'] == i)].groupby(ctd_n['결제ID'].dt.hour)['상품별 합계'].sum()
    # 시간과 매출 컬럼으로 데이터프레임 생성 후 딕셔너리에 저장
    ctd_dict3[var_name] = pd.DataFrame({'시간': monthly_sales.index, '매출': monthly_sales.values})

# 결과 출력
for key, value in ctd_dict3.items():
    print(f'{key} 매출:\n{value}')


In [ ]:
# 2022년도 월별 시간대별 매출평균
ctd_dict3m = {}

for i in range(2, 13):
    var_name = f'ctd_22s{i}'
    # '결제ID' 컬럼의 데이터를 datetime 형식으로 변환하여 시간 정보 추출 후 그룹화
    ctd_n['결제ID'] = pd.to_datetime(ctd_n['결제ID'])
    monthly_sales = ctd_n[(ctd_n['연'] == 2022) & (ctd_n['월'] == i)].groupby(ctd_n['결제ID'].dt.hour)['상품별 합계'].sum()
    # 시간과 매출 컬럼으로 데이터프레임 생성 후 딕셔너리에 저장 (평균을 int로 변환하여 저장)
    ctd_dict3m[var_name] = pd.DataFrame({'시간': monthly_sales.index, '매출': monthly_sales.values.astype(int)})

# 결과 출력
for key, value in ctd_dict3m.items():
    print(f'{key} 매출:\n{value}')


In [ ]:
# 2023년도 월별 시간대별 매출합계
ctd_dict4 = {}

for i in range(1, 6):
    var_name = f'ctd_23s{i}'
    # '결제ID' 컬럼의 데이터를 datetime 형식으로 변환하여 시간 정보 추출 후 그룹화
    ctd_n['결제ID'] = pd.to_datetime(ctd_n['결제ID'])
    monthly_sales = ctd_n[(ctd_n['연'] == 2023) & (ctd_n['월'] == i)].groupby(ctd_n['결제ID'].dt.hour)['상품별 합계'].sum()
    # 시간과 매출 컬럼으로 데이터프레임 생성 후 딕셔너리에 저장
    ctd_dict4[var_name] = pd.DataFrame({'시간': monthly_sales.index, '매출': monthly_sales.values})

# 결과 출력
for key, value in ctd_dict4.items():
    print(f'{key} 매출:\n{value}')


In [ ]:
# 22년 6,7,8,12월 23년 1,2월 입도객 수
months = dv['월'].unique()

for month in months:
    data_month = dv[dv['월'] == month]
    pivot_table = data_month.pivot_table(index='요일', values='일별 입도객', aggfunc='sum')

    pivot_table = pivot_table.sort_values(by='일별 입도객', ascending=False)

    plt.figure(figsize=(10, 6))
    plt.bar(pivot_table.index, pivot_table['일별 입도객'], color='#F1CE63', width=0.7)

    # 요일의 첫 글자로 xlabel 설정
    plt.xlabel('요일', fontsize=10)
    plt.xticks(range(len(pivot_table.index)), [day[0] for day in pivot_table.index], fontsize=10)  # 첫 글자만 사용

    plt.ylabel('입도객 수', fontsize=10)
    plt.title(f'{month}월 요일별 입도객 수(합계)', fontsize=10)

    # 막대 위에 값을 표시
    for index, value in enumerate(pivot_table['일별 입도객']):
        plt.annotate(f'{value}', xy=(index, value), ha='center', va='bottom', fontsize=10)

    plt.xticks(fontsize=10)
    plt.yticks(fontsize=10)
    plt.tight_layout()

    for spine in plt.gca().spines.values():
        spine.set_color('black')
    plt.gca().set_facecolor('white')

    plt.show()


In [ ]:
# 22년 각 월별 상위 10개 상품별 주문수 그래프 그리기
for i in range(2, 13):
    var_name = f'ctd_22{i}'
    sorted_data = ctd_dict[var_name].sort_values(by=f'{i}월_주문수', ascending=True)
    plt.figure(figsize=(10, 6))
    plt.barh(sorted_data[f'{i}월_상품명'], sorted_data[f'{i}월_주문수'], color='#F1CE63')  # x-axis and y-axis are swapped
    plt.xlabel('주문수', fontsize=10)
    plt.ylabel('상품명', fontsize=10)
    plt.title(f'22년 {i}월 주문수 상위 10개 상품명', fontsize=10)  # title

    # 막대 위에 값을 표시
    for index, value in enumerate(sorted_data[f'{i}월_주문수']):
        plt.text(value, index, str(value), ha='left', va='center', fontsize=8)

    plt.xticks(fontsize=10)
    plt.yticks(fontsize=10)
    plt.tight_layout()

    for spine in plt.gca().spines.values():
        spine.set_color('black')
    plt.gca().set_facecolor('white')

    plt.show()

In [ ]:
# 23년 각 월별 상위 10개 상품별 주문수 그래프 그리기
for i in range(1, 6):
    var_name = f'ctd_23{i}'
    sorted_data = ctd_dict1[var_name].sort_values(by=f'{i}월_주문수', ascending=True)  # 오름차순으로 정렬
    plt.figure(figsize=(10, 6))
    plt.barh(sorted_data[f'{i}월_상품명'], sorted_data[f'{i}월_주문수'], color='#F1CE63')  # x-axis and y-axis are swapped
    plt.xlabel('주문수', fontsize=10)  # x-axis label
    plt.ylabel('상품명', fontsize=10)  # y-axis label
    plt.title(f'23년 {i}월 주문수 상위 10개 상품명', fontsize=10)  # title
    plt.tight_layout()

    plt.xticks(fontsize=10)  # x축 눈금 레이블 글씨 크기 조정
    plt.yticks(fontsize=10)  # y축 눈금 레이블 글씨 크기 조정


    for spine in plt.gca().spines.values():
        spine.set_color('black')
    plt.gca().set_facecolor('white')

    # 막대 위에 값을 표시하는 로직 추가
    for index, value in enumerate(sorted_data[f'{i}월_주문수']):
        plt.text(value, index, str(value), ha='left', va='center', fontsize=8)

    plt.show()


In [ ]:
# 각 시간별 상위 10개 상품별 주문수 그래프 그리기
for i in range(7, 19):
    var_name = f'ctd_h_{i}'
    sorted_data = ctd_dict2[var_name].sort_values(by=f'{i}시_주문수', ascending=True)  # 오름차순으로 정렬
    plt.figure(figsize=(10, 6))
    plt.barh(sorted_data[f'{i}시_상품명'], sorted_data[f'{i}시_주문수'], color='#F1CE63')  # x-axis and y-axis are swapped
    plt.xlabel('주문수', fontsize=10)  # x-axis label
    plt.ylabel('상품명', fontsize=10)  # y-axis label
    plt.title(f'{i}시 주문수 상위 10개 상품명', fontsize=10)  # title
    plt.tight_layout()

    plt.xticks(fontsize=8)  # x축 눈금 레이블 글씨 크기 조정
    plt.yticks(fontsize=10)  # y축 눈금 레이블 글씨 크기 조정

    for spine in plt.gca().spines.values():
        spine.set_color('black')
    plt.gca().set_facecolor('white')

    # 막대 위에 값을 표시하는 로직 추가
    for index, value in enumerate(sorted_data[f'{i}시_주문수']):
        plt.text(value, index, str(value), ha='left', va='center', fontsize=8)

    plt.show()

In [ ]:
# 22년 시간대별 매출 그래프
for i in range(2, 13):
    var_name = f'ctd_22s{i}'
    sorted_data = ctd_dict3[var_name].sort_values(by=['시간'], ascending=True)
    plt.figure(figsize=(10, 6))
    plt.bar(sorted_data['시간'], sorted_data['매출'], color='#F1CE63', width=0.7)
    plt.xlabel('시간', fontsize=10)
    plt.ylabel('매출', fontsize=10)
    plt.title(f'22년 {i}월 시간대별 매출', fontsize=10)
    plt.ticklabel_format(style='plain')

    for index, value in enumerate(sorted_data['매출']):
        plt.text(sorted_data['시간'].iloc[index], value, str(value), ha='center', va='bottom')

    plt.xticks(fontsize=10)
    plt.yticks(fontsize=10)

    for spine in plt.gca().spines.values():
        spine.set_color('black')
    plt.gca().set_facecolor('white')

    plt.show()

In [ ]:
# 23년 시간대별 매출 그래프
for i in range(1, 6):
    var_name = f'ctd_23s{i}'
    sorted_data = ctd_dict4[var_name].sort_values(by=['시간'], ascending=True)
    plt.figure(figsize=(10, 6))
    plt.bar(sorted_data['시간'], sorted_data['매출'], color='#F1CE63', width=0.7)
    plt.xlabel('시간', fontsize=10)
    plt.ylabel('매출', fontsize=10)
    plt.title(f'23년 {i}월 시간대별 매출', fontsize=10)
    plt.ticklabel_format(style='plain')

    # 막대 위에 값을 표시하는 로직 추가
    for index, value in enumerate(sorted_data['매출']):
        plt.text(sorted_data['시간'].iloc[index], value, str(value), ha='center', va='bottom')

    plt.xticks(fontsize=10)
    plt.yticks(fontsize=10)

    # Set black border with a white background
    for spine in plt.gca().spines.values():
        spine.set_color('black')
    plt.gca().set_facecolor('white')

    plt.show()

In [ ]:
# 월별 주문수 상위 10개 상품명 추출
ctd_moc = ctd_n.groupby(['연', '월'])['상품명'].value_counts().groupby(level=[0, 1]).nlargest(10)

# MultiIndex를 제거하여 데이터프레임으로 변환하고 열 이름을 재지정
ctd_moc = ctd_moc.reset_index(level=[0, 1], drop=True).reset_index(name='주문수')

# 결과 출력
print(ctd_moc)

In [ ]:
# 엑셀 파일로 저장
ctd_moc.to_excel('월별 주문수 상위 10 상품명.xlsx', index=False)

In [ ]:
# 날짜별 매출과 관광객 그래프

# 이중 축을 가진 그림과 서브플롯 생성
fig, ax1 = plt.subplots(figsize=(12, 8)) # figsize로 그래프의 크기 조정

# 첫 번째 축에 날짜별 매출 그래프 그리기
ax1.plot(yms_v['날짜'].dt.strftime('%y/%m'), yms_v['매출'], label='날짜별 매출', marker='o', color='tab:red')
ax1.set_xlabel('날짜', fontsize=14) # fontsize로 x축 레이블의 크기 조정
ax1.set_ylabel('매출', color='tab:red', fontsize=14) # fontsize로 y축 레이블의 크기 조정
ax1.tick_params(axis='y', labelcolor='tab:red', labelsize=12) # labelsize로 y축 눈금의 크기 조정

# 두 번째 축 생성
ax2 = ax1.twinx()

# 두 번째 축에 날짜별 관광객 그래프 그리기
ax2.plot(yms_v['날짜'].dt.strftime('%y/%m'), yms_v['관광객'], label='날짜별 관광객', marker='x', color='tab:blue')
ax2.set_ylabel('관광객 수', color='tab:blue', fontsize=14) # fontsize로 y축 레이블의 크기 조정
ax2.tick_params(axis='y', labelcolor='tab:blue', labelsize=12) # labelsize로 y축 눈금의 크기 조정

# y축 격자 표시
ax2.grid(axis='y')

# x축 눈금 표시 간격 설정 (1달 간격으로 눈금 표시) ???
ax1.tick_params(axis='x', rotation=45, labelsize=12) # fontsize로 x축 눈금의 크기 조정

# x축 눈금의 크기를 8개로 제한
ax1.xaxis.set_major_locator(plt.MaxNLocator(10))

# 그래프 제목과 범례 설정
plt.title('날짜별 매출과 관광객 수', fontsize=16) # fontsize로 제목의 크기 조정

# 격자 표시
ax1.grid(True)

plt.tight_layout()
plt.show()

In [ ]:
# 날짜별 주문건수와 관광객 그래프

# 이중 축을 가진 그림과 서브플롯 생성
fig, ax1 = plt.subplots(figsize=(12, 8)) # figsize로 그래프의 크기 조정

# 첫 번째 축에 날짜별 주문수 그래프 그리기
ax1.plot(ymoc_v['날짜'].dt.strftime('%y/%m'), ymoc_v['주문수'], label='날짜별 주문수', marker='o', color='tab:red')
ax1.set_xlabel('날짜', fontsize=14) # fontsize로 x축 레이블의 크기 조정
ax1.set_ylabel('주문수', color='tab:red', fontsize=14) # fontsize로 y축 레이블의 크기 조정
ax1.tick_params(axis='y', labelcolor='tab:red', labelsize=12) # labelsize로 y축 눈금의 크기 조정

# 두 번째 축 생성
ax2 = ax1.twinx()

# 두 번째 축에 날짜별 관광객 그래프 그리기
ax2.plot(ymoc_v['날짜'].dt.strftime('%y/%m'), ymoc_v['관광객'], label='날짜별 관광객', marker='x', color='tab:blue')
ax2.set_ylabel('관광객 수', color='tab:blue', fontsize=14) # fontsize로 y축 레이블의 크기 조정
ax2.tick_params(axis='y', labelcolor='tab:blue', labelsize=12) # labelsize로 y축 눈금의 크기 조정

# y축 격자 표시
ax2.grid(axis='y')

# x축 눈금 표시 간격 설정 (1달 간격으로 눈금 표시) ???
ax1.tick_params(axis='x', rotation=45, labelsize=12) # fontsize로 x축 눈금의 크기 조정

# x축 눈금의 크기를 8개로 제한
ax1.xaxis.set_major_locator(plt.MaxNLocator(10))

# 그래프 제목과 범례 설정
plt.title('날짜별 주문수와 관광객 수', fontsize=16) # fontsize로 제목의 크기 조정

# 격자 표시
ax1.grid(True)

plt.tight_layout()
plt.show()

In [ ]:
# 해외관광객 제주관광객 수 비교

# 이중 축을 가진 그림과 서브플롯 생성
fig, ax1 = plt.subplots(figsize=(12, 8)) # figsize로 그래프의 크기 조정

# 첫 번째 축에 날짜별 해외 그래프 그리기
ax1.plot(v_ta['날짜'].dt.strftime('%y/%m'), v_ta['해외관광객'], label='날짜별 주문수', marker='o', color='tab:red')
ax1.set_xlabel('날짜', fontsize=14) # fontsize로 x축 레이블의 크기 조정
ax1.set_ylabel('해외관광객 수', color='tab:red', fontsize=14) # fontsize로 y축 레이블의 크기 조정
ax1.tick_params(axis='y', labelcolor='tab:red', labelsize=12) # labelsize로 y축 눈금의 크기 조정

# 두 번째 축 생성
ax2 = ax1.twinx()

# 두 번째 축에 날짜별 관광객 그래프 그리기
ax2.plot(v_ta['날짜'].dt.strftime('%y/%m'), v_ta['관광객'], label='날짜별 관광객', marker='x', color='tab:blue')
ax2.set_ylabel('관광객 수', color='tab:blue', fontsize=14) # fontsize로 y축 레이블의 크기 조정
ax2.tick_params(axis='y', labelcolor='tab:blue', labelsize=12) # labelsize로 y축 눈금의 크기 조정

# y축 격자 표시
ax2.grid(axis='y')

# x축 눈금 표시 간격 설정 (1달 간격으로 눈금 표시) ???
ax1.tick_params(axis='x', rotation=45, labelsize=12) # fontsize로 x축 눈금의 크기 조정

# x축 눈금의 크기를 8개로 제한
ax1.xaxis.set_major_locator(plt.MaxNLocator(10))

# 그래프 제목과 범례 설정
plt.title('월별 해외관광객 수 와 제주관광객 수', fontsize=16) # fontsize로 제목의 크기 조정

# 격자 표시
ax1.grid(True)

plt.tight_layout()
plt.show()

In [ ]:
# 매출과 제주관광객의 소비 비교

# 이중 축을 가진 그림과 서브플롯 생성
fig, ax1 = plt.subplots(figsize=(12, 8)) # figsize로 그래프의 크기 조정

# 첫 번째 축에 월별 매출 그리기
ax1.plot(yms_pc['기준연월'].dt.strftime('%y/%m'), yms_pc['매출'], label='월별 매출', marker='o', color='tab:red')
ax1.set_xlabel('기준연월', fontsize=14) # fontsize로 x축 레이블의 크기 조정
ax1.set_ylabel('매출', color='tab:red', fontsize=14) # fontsize로 y축 레이블의 크기 조정
ax1.tick_params(axis='y', labelcolor='tab:red', labelsize=12) # labelsize로 y축 눈금의 크기 조정

# 두 번째 축 생성
ax2 = ax1.twinx()

# 두 번째 축에 월별 제주관광객 전체업종소비액 그래프 그리기
ax2.plot(yms_pc['기준연월'].dt.strftime('%y/%m'), yms_pc['전체업종소비액'], label='월별 소비금액', marker='x', color='tab:blue')
ax2.set_ylabel('전체업종소비금액', color='tab:blue', fontsize=14) # fontsize로 y축 레이블의 크기 조정
ax2.tick_params(axis='y', labelcolor='tab:blue', labelsize=12) # labelsize로 y축 눈금의 크기 조정

# y축 격자 표시
ax2.grid(axis='y')

# x축 눈금 표시 간격 설정 (1달 간격으로 눈금 표시) ???
ax1.tick_params(axis='x', rotation=45, labelsize=12) # fontsize로 x축 눈금의 크기 조정

# x축 눈금의 크기를 8개로 제한
ax1.xaxis.set_major_locator(plt.MaxNLocator(10))

# 그래프 제목과 범례 설정
plt.title('월별 매출과 제주관광객 전체업종 소비금액', fontsize=16) # fontsize로 제목의 크기 조정

# 격자 표시
ax1.grid(True)

plt.tight_layout()
plt.show()

In [ ]:
# 매출과 제주관광객의 소비 비교

# 이중 축을 가진 그림과 서브플롯 생성
fig, ax1 = plt.subplots(figsize=(12, 8)) # figsize로 그래프의 크기 조정

# 첫 번째 축에 월별 매출 그리기
ax1.plot(yms_pc['기준연월'].dt.strftime('%y/%m'), yms_pc['매출'], label='월별 매출', marker='o', color='tab:red')
ax1.set_xlabel('기준연월', fontsize=14) # fontsize로 x축 레이블의 크기 조정
ax1.set_ylabel('매출', color='tab:red', fontsize=14) # fontsize로 y축 레이블의 크기 조정
ax1.tick_params(axis='y', labelcolor='tab:red', labelsize=12) # labelsize로 y축 눈금의 크기 조정

# 두 번째 축 생성
ax2 = ax1.twinx()

# 두 번째 축에 월별 제주관광객 식음료업소비액 그래프 그리기
ax2.plot(yms_pc['기준연월'].dt.strftime('%y/%m'), yms_pc['식음료업소비액'], label='월별 소비금액', marker='x', color='tab:blue')
ax2.set_ylabel('식음료업종소비금액', color='tab:blue', fontsize=14) # fontsize로 y축 레이블의 크기 조정
ax2.tick_params(axis='y', labelcolor='tab:blue', labelsize=12) # labelsize로 y축 눈금의 크기 조정

# y축 격자 표시
ax2.grid(axis='y')

# x축 눈금 표시 간격 설정 (1달 간격으로 눈금 표시) ???
ax1.tick_params(axis='x', rotation=45, labelsize=12) # fontsize로 x축 눈금의 크기 조정

# x축 눈금의 크기를 8개로 제한
ax1.xaxis.set_major_locator(plt.MaxNLocator(10))

# 그래프 제목과 범례 설정
plt.title('월별 매출과 제주관광객 식음료업 소비금액', fontsize=16) # fontsize로 제목의 크기 조정

# 격자 표시
ax1.grid(True)

plt.tight_layout()
plt.show()

In [ ]:
# 차트레이스 만들기
!pip install bar_chart_race
import bar_chart_race as bcr

data = pd.DataFrame({
    '날짜': ['2022년 2월', '2022년 3월', '2022년 4월', '2022년 5월', '2022년 6월', '2022년 7월', '2022년 8월', '2022년 9월', '2022년 10월', '2022년 11월', '2022년 12월', '2023년 1월', '2023년 2월', '2023년 3월', '2023년 4월', '2023년 5월'],
    '(I) 아메리카노_쥬시': [0, 257, 327, 311, 290, 279, 308, 181, 0, 0, 0, 0, 0, 200, 167, 21],
    '(I) 아메리카노_클래식': [202, 589, 871, 832, 722, 587, 620, 547, 484, 419, 218, 243, 268, 454, 462, 553],
    '(I) 카페라떼_클래식': [0, 0, 0, 0, 0, 0, 0, 170, 0, 0, 0, 0, 0, 0, 0, 0],
    '무화과_휘낭시에': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 155, 0, 202, 208],
    '바스크_치즈케이크': [146, 336, 404, 396, 319, 222, 212, 246, 297, 265, 219, 248, 223, 288, 219, 264],
    '브라우니': [139, 325, 403, 343, 280, 250, 205, 241, 292, 233, 177, 211, 189, 257, 249, 254],
    '슈퍼클린_쥬시': [219, 453, 428, 334, 341, 295, 284, 211, 189, 166, 156, 155, 0, 0, 0, 0],
    '슈퍼클린_클래식': [257, 534, 871, 381, 394, 329, 310, 285, 252, 249, 322, 290, 177, 0, 0, 193],
    '슈퍼클린_싱글': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 210, 178, 0, 294, 0, 0],
    '싱글_슈퍼클린': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 321, 0],
    '아메리카노_클래식': [184, 396, 359, 255, 0, 0, 0, 0, 291, 305, 288, 291, 248, 349, 260, 238],
    '아이스_유자_아메리카노': [292, 725, 1005, 914, 821, 672, 757, 550, 467, 467, 375, 420, 514, 821, 739, 817],
    '아이스_텐저린_라떼': [555, 1494, 2197, 1769, 1402, 1136, 1244, 896, 933, 755, 627, 678, 835, 1351, 1356, 1432],
    '제주_유기농_감귤주스': [0, 0, 337, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    '제주_유기농_귤피주스': [0, 0, 0, 0, 235, 274, 266, 0, 187, 0, 0, 0, 148, 186, 0, 197],
    '카페_루이지': [144, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    '텐저린카푸치노': [822, 781, 0, 552, 550, 380, 415, 533, 742, 849, 1045, 1062, 1050, 288, 0, 0],
    '플레인_휘낭시에': [0, 0, 0, 0, 0, 0, 0, 0, 0, 281, 0, 0, 0, 0, 176, 0]
})
# '날짜' 열을 인덱스로 설정합니다
data.set_index('날짜', inplace=True)

# 애니메이션 라인 차트 레이스를 생성합니다
bcr.bar_chart_race(data, n_bars=10, steps_per_period=10)


# 비식별화

In [ ]:
# 22년 시간대별 매출
for i in range(2, 13):
    var_name = f'ctd_22s{i}'
    sorted_data = ctd_dict3[var_name].sort_values(by=['시간'], ascending=True)
    plt.figure(figsize=(10, 6))
    plt.bar(sorted_data['시간'], sorted_data['매출'], color='#F1CE63', width=0.7)
    plt.xlabel('시간', fontsize=10)
    plt.ylabel('매출', fontsize=10)
    plt.title('시간대별 매출', fontsize=10)
    plt.ticklabel_format(style='plain')

    plt.xticks([])
    plt.yticks([])  # y축 값 표시 제거

    for spine in plt.gca().spines.values():
        spine.set_color('black')
    plt.gca().set_facecolor('white')

    plt.show()

In [ ]:
# 23년 시간대별 매출
for i in range(1, 6):
    var_name = f'ctd_23s{i}'
    sorted_data = ctd_dict4[var_name].sort_values(by=['시간'], ascending=True)
    plt.figure(figsize=(10, 6))
    plt.bar(sorted_data['시간'], sorted_data['매출'], color='#F1CE63', width=0.7)
    plt.xlabel('시간', fontsize=10)
    plt.ylabel('매출', fontsize=10)
    plt.title('시간대별 매출', fontsize=10)
    plt.ticklabel_format(style='plain')

    plt.xticks([])
    plt.yticks([])

    # Set black border with a white background
    for spine in plt.gca().spines.values():
        spine.set_color('black')
    plt.gca().set_facecolor('white')

    plt.show()

In [ ]:
# 22년 각 월별 상위 10개 상품별 주문수 그래프 그리기
for i in range(2, 13):
    var_name = f'ctd_22{i}'
    sorted_data = ctd_dict[var_name].sort_values(by=f'{i}월_주문수', ascending=True)
    plt.figure(figsize=(10, 6))
    plt.barh(sorted_data[f'{i}월_상품명'], sorted_data[f'{i}월_주문수'], color='#F1CE63')  # x-axis and y-axis are swapped
    plt.xlabel('주문수', fontsize=10)
    plt.ylabel('상품명', fontsize=10)
    plt.title('주문수 상위 10개 상품명', fontsize=10)  # title

    # 막대 위에 값을 표시
    plt.xticks([])
    plt.yticks([])
    plt.tight_layout()

    for spine in plt.gca().spines.values():
        spine.set_color('black')
    plt.gca().set_facecolor('white')

    plt.show()

In [ ]:
# 23년 각 월별 상위 10개 상품별 주문수 그래프 그리기
for i in range(1, 6):
    var_name = f'ctd_23{i}'
    sorted_data = ctd_dict1[var_name].sort_values(by=f'{i}월_주문수', ascending=True)  # 오름차순으로 정렬
    plt.figure(figsize=(10, 6))
    plt.barh(sorted_data[f'{i}월_상품명'], sorted_data[f'{i}월_주문수'], color='#F1CE63')  # x-axis and y-axis are swapped
    plt.xlabel('주문수', fontsize=10)  # x-axis label
    plt.ylabel('상품명', fontsize=10)  # y-axis label
    plt.title('주문수 상위 10개 상품명', fontsize=10)  # title
    plt.tight_layout()

    plt.xticks([])  # x축 눈금 레이블 글씨 크기 조정
    plt.yticks([])  # y축 눈금 레이블 글씨 크기 조정

    for spine in plt.gca().spines.values():
        spine.set_color('black')
    plt.gca().set_facecolor('white')

    plt.show()
